<span style='font-size:150%'>딥러닝 수행 (10점) - TensorFlow, Keras를 활용하여 딥러닝 수행</span>   
  
다양한 방식으로 딥러닝을 수행한 코딩을 모두 제출할 것    
   
파일명 : 4_Deep learning_1.ipynb   
   
(여러개의 파일을 사용한 경우 Deep learning_2.ipynb, Deep learning_3.ipynb...)   

In [1]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import matplotlib.font_manager

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

seed=0
np.random.seed(3)
tf.random.set_seed(3)

In [2]:
import os
os.getcwd()

'D:\\ai\\Final'

# 데이터
Data_set_1 : 종가,시가, 고가, 저가, 거래량, 변동 데이터가 담김. 아직 클래스 데이터 문자 형태  
Data_set_2 : 기존 속성 데이터들을 속성들 간의 차로 변경. 스케일링됨. 클래스 데이터 숫자 형태  

In [3]:
# 데이터 불러오기

# 일반 데이터
data_set=pd.read_csv("D:/ai/Final/Data set_2.csv")

In [4]:
data_set

,close-start,high-low,high-close,high-start,close-low,start-low,volume,Diff
0,0.541985,0.033708,0.067568,0.084337,0.051282,0.024390,0.111197,0
1,0.465649,0.168539,0.121622,0.012048,0.153846,0.243902,0.123828,1
2,0.465649,0.112360,0.202703,0.084337,0.012821,0.109756,0.134799,1
3,0.633588,0.101124,0.000000,0.168675,0.192308,0.012195,0.129628,0
4,0.412214,0.112360,0.202703,0.000000,0.012821,0.195122,0.059433,1
...,...,...,...,...,...,...,...,...
594,0.603053,0.292135,0.162162,0.265060,0.256410,0.121951,0.437442,0
595,0.450382,0.224719,0.324324,0.168675,0.025641,0.146341,0.374049,1
596,0.801527,0.359551,0.000000,0.433735,0.487179,0.024390,0.357297,0
597,0.557252,0.224719,0.243243,0.265060,0.102564,0.048780,0.644365,1


# 데이터셋 설정

주가를 예측하는 것은 시계열 데이터를 필요로 하기 때문에 기존 데이터를 시계열 데이터에 맞게 변형을 해줘야 한다.  
며칠 동안의 데이터를 가지고 예측을 할지를 window_size에 입력받고, 해당 숫자를 기반으로 시계열 데이터를 생성할 것이다.  
window_size에 따라 딥러닝 결과가 달라질 수 있기 때문에 시계열 데이터를 '2_...'나 '3_...'에서 만들지 않고 '4_...'에서 바로바로 만들어서 사용했다.

또한 여러번 실행을 해보기 위해 함수를 사용했다.

In [5]:
# window의 사이즈와 train dataset의 사이즈 입력받는 함수

def set_window_train_size(data):
    
    # 학습 데이터셋 70% 테스트 데이터셋 30%로 나눈다. (고정)
    train_size=int(len(data)*0.7) 
    
    # 시계열 데이터셋을 만들 때, 몇 개의 데이터를 가지고 만들지 설정한다.
    # 예 : window_size=30 => 30일간의 데이터를 가진 시계열 데이터셋
    window_size=int(input("window size : "))

    return window_size, train_size

In [6]:
# feature 데이터와 class 데이터를 구분

def split_feature_class(data):
    
    #feature_cols = ['close-start', 'high-low', 'high-close', 'high-start','close-low','start-low','volume']
    #close_cols = ['close']
    
    feature_cols = ['close-start', 'high-low', 'high-close', 'high-start','close-low','start-low','volume']
    close_cols = ['Diff']
    
    data_feature = data[feature_cols]
    data_close = data[close_cols]
    
    return data_feature, data_close

In [7]:
# window_size에 따라 데이터를 묶어주고 새로운 데이터셋 리턴
def make_window_data(feature_data, class_data, window_size):
    
    feature_list=[]
    class_list=[]
    
    for i in range(len(feature_data)-window_size):
        feature_list.append(np.array(feature_data.iloc[i:i+window_size]))
        class_list.append(np.array(class_data.iloc[i+window_size]))
    
    return np.array(feature_list), np.array(class_list)

In [8]:
def make_dataset(data):
    
    # 시계열 데이터 크기(window_size)와 학습 데이터(train_size)의 크기
    window_size, train_size=set_window_train_size(data)
    
    # 학습 데이터셋과 테스트 데이터셋 분리
    train = data[:train_size] 
    test = data[train_size:] 
    
    train_feature, train_class = split_feature_class(train)
    test_feature, test_class = split_feature_class(test)
    
    train_feature, train_class = make_window_data(train_feature, train_class, window_size)
    test_feature, test_class = make_window_data(test_feature, test_class, window_size)
    
    return train_feature, train_class, test_feature, test_class, window_size

In [9]:
# 학습 데이터셋을 train과 valid로 분리

def train_valid(train_feature,train_class):
    split_size= float(input("학습 데이터 중 test size(validation data의 size)의 비율 (예 : 0.2) : "))
    X_train, X_valid, Y_train, Y_valid = train_test_split(train_feature, train_class, test_size = split_size, random_state=seed)
    return X_train, X_valid, Y_train, Y_valid

# 기본 데이터에 대한 딥러닝

In [10]:
train_feature, train_class, test_feature, test_class, window_size = make_dataset(data_set)

X_train, X_valid, Y_train, Y_valid = train_valid(train_feature, train_class)

window size : 3
학습 데이터 중 test size(validation data의 size)의 비율 (예 : 0.2) : 0.2


<span style='font-size:150%'>[LSTM]</span>  
  
시계열 데이터이기 때문에 LSTM을 사용했다.  
2진 class이기 때문에  활성화 함수로 'sigmoid'를, 손실 함수로 'binary_crossentropy'를 사용한다.

In [21]:
model = Sequential()
model.add(LSTM(20, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), #시계열 데이터의 구조
               activation='relu',return_sequences=False))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

#optimizer='rmsprop'
#optimizer='adam'

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20)                2240      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 2,261
Trainable params: 2,261
Non-trainable params: 0
_________________________________________________________________


# 모델 업데이트 과정 저장을 위한 환경 설정

In [22]:
# 모델 저장 폴더 생성
model_dir='./model_save'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
# 파일 저장 경로
model_path='./model_save/12_'+str(window_size)+'_{epoch:d}_{val_loss:.4f}_{val_accuracy:.4f}.h5'

In [23]:
model_path

'./model_save/12_3_{epoch:d}_{val_loss:.4f}_{val_accuracy:.4f}.h5'

In [24]:
# 학습 자동 중단 설정
# 20번 연속 성능에 변화 없으면 학습 멈춤
early_stop = EarlyStopping(monitor='val_loss', patience=20)

In [25]:
# 모델 업데이트 및 저장
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

In [26]:
history = model.fit(X_train, Y_train,                     
                    validation_data=(X_valid, Y_valid), 
                    epochs=200, 
                    batch_size=20, 
                    callbacks=[early_stop, checkpoint])

Train on 332 samples, validate on 84 samples
Epoch 1/200
320/332 [===========================>..] - ETA: 0s - loss: 0.6927 - accuracy: 0.5250 
Epoch 00001: val_loss improved from inf to 0.69424, saving model to ./model_save/12_3_1_0.6942_0.4762.h5
332/332 [==============================] - 1s 4ms/sample - loss: 0.6926 - accuracy: 0.5271 - val_loss: 0.6942 - val_accuracy: 0.4762
Epoch 2/200
320/332 [===========================>..] - ETA: 0s - loss: 0.6923 - accuracy: 0.5188
Epoch 00002: val_loss improved from 0.69424 to 0.69414, saving model to ./model_save/12_3_2_0.6941_0.4762.h5
332/332 [==============================] - 0s 261us/sample - loss: 0.6923 - accuracy: 0.5181 - val_loss: 0.6941 - val_accuracy: 0.4762
Epoch 3/200
 20/332 [>.............................] - ETA: 0s - loss: 0.6937 - accuracy: 0.4500
Epoch 00003: val_loss did not improve from 0.69414
332/332 [==============================] - 0s 207us/sample - loss: 0.6922 - accuracy: 0.5241 - val_loss: 0.6943 - val_accuracy: 0.

Epoch 55/200
240/332 [====================>.........] - ETA: 0s - loss: 0.6798 - accuracy: 0.5750
Epoch 00055: val_loss did not improve from 0.69035
332/332 [==============================] - 0s 348us/sample - loss: 0.6857 - accuracy: 0.5602 - val_loss: 0.6920 - val_accuracy: 0.5000
Epoch 56/200
220/332 [==================>...........] - ETA: 0s - loss: 0.6861 - accuracy: 0.5636
Epoch 00056: val_loss did not improve from 0.69035
332/332 [==============================] - 0s 348us/sample - loss: 0.6853 - accuracy: 0.5572 - val_loss: 0.6905 - val_accuracy: 0.4881
Epoch 57/200
200/332 [=================>............] - ETA: 0s - loss: 0.6882 - accuracy: 0.5600
Epoch 00057: val_loss did not improve from 0.69035
332/332 [==============================] - 0s 363us/sample - loss: 0.6858 - accuracy: 0.5572 - val_loss: 0.6913 - val_accuracy: 0.4881
Epoch 58/200
240/332 [====================>.........] - ETA: 0s - loss: 0.6822 - accuracy: 0.5708
Epoch 00058: val_loss did not improve from 0.69035

In [27]:
history

In [28]:
print("\nAccuracy : %.4f"%(model.evaluate(train_feature,train_class)[1]))

416/416 [==============================] - 0s 963us/sample - loss: 0.6847 - accuracy: 0.5457

Accuracy : 0.5457


In [29]:
print("\nAccuracy : %.4f"%(model.evaluate(test_feature,test_class)[1]))

177/177 [==============================] - 0s 81us/sample - loss: 0.7084 - accuracy: 0.4689

Accuracy : 0.4689


탈락.

=====================================================================================================================
전반적으로 다음과 같은 결과를 보였다.  

5일 > 10일 > 3일  

window_size와 batch_size를 다르게 한 12개의 모델에 대한 딥러닝을 진행했다. 그 중에 후보는 단 2개 뿐이었다.

[후보1]  

window_size = 5

batch_size= 10

학습 데이터셋 : Accuracy : 0.5894

테스트 데이터셋 : Accuracy : 0.5486

[후보 2]  

window_size = 5

batch_size = 5

학습 데이터셋 : Accuracy : 0.5845

테스트 데이터셋 : Accuracy : 0.5486

둘다 테스트 데이터셋에 대한 정확도는 같다.  
후보 1이 후보 2보다 학습 데이터셋에 대한 정확도가 높으므로 후보 1을 선택해서 5_에서 분석 진행